In [14]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as PathEffects
import matplotlib.dates as mdates
import matplotlib.ticker as mtick

# --- 1. THIẾT LẬP ---
sns.set_theme(style="whitegrid")
BASE_DIR = '..'
PROCESSED_DIR = os.path.join(BASE_DIR, 'processed', 'brussels')
FIGURES_DIR = os.path.join(BASE_DIR, 'figures')
os.makedirs(FIGURES_DIR, exist_ok=True)
CITY_SUFFIX = 'brussels'

print("--- BẮT ĐẦU VẼ CHUYÊN SÂU CHO BRUSSELS ---")

# --- 2. NẠP DỮ LIỆU KPI ---
try:
    # Load các file KPI đã tạo
    kpi_supply = pd.read_csv(os.path.join(PROCESSED_DIR, f'kpi_summary_general_{CITY_SUFFIX}.csv'))
    kpi_room = pd.read_csv(os.path.join(PROCESSED_DIR, f'kpi_room_type_{CITY_SUFFIX}.csv'))
    kpi_season = pd.read_csv(os.path.join(PROCESSED_DIR, f'kpi_seasonality_{CITY_SUFFIX}.csv'))

    # Hàm sắp xếp thời gian
    def sort_by_date(df, date_col='snapshot_date'):
        df['dt'] = pd.to_datetime(df[date_col], errors='coerce')
        return df.sort_values('dt')

    kpi_supply = sort_by_date(kpi_supply)
    # kpi_room và kpi_season sẽ xử lý riêng khi vẽ
    print("--- ĐANG NẠP DỮ LIỆU TỔNG HỢP ---")
except FileNotFoundError as e:
    print(f"❌ Lỗi: Không tìm thấy file KPI. Chi tiết: {e}")
    kpi_supply = pd.DataFrame() # Tạo dataframe rỗng để code không crash

# --- 3. NẠP DỮ LIỆU CHI TIẾT (LATEST SNAPSHOT) ---
from datetime import datetime

def parse_snapshot_name(name):
    """
    Parse tên thư mục snapshot dạng '21 June, 2025' thành datetime.
    Trả về None nếu parse thất bại.
    """
    try:
        return datetime.strptime(name.strip(), "%d %B, %Y")
    except:
        return None

df_detail = pd.DataFrame()
LATEST_SNAPSHOT = "Unknown"

if os.path.exists(PROCESSED_DIR):

    # Lấy danh sách thư mục + chỉ giữ thư mục có format date hợp lệ
    raw_snaps = [
        d for d in os.listdir(PROCESSED_DIR)
        if os.path.isdir(os.path.join(PROCESSED_DIR, d))
    ]

    # Parse + giữ lại snapshot hợp lệ
    snapshots = [s for s in raw_snaps if parse_snapshot_name(s) is not None]

    # Sort theo ngày (mới nhất trước)
    snapshots = sorted(
        snapshots,
        key=lambda x: parse_snapshot_name(x),
        reverse=True
    )

    # Lấy file mới nhất
    for snap in snapshots:
        file_path = os.path.join(PROCESSED_DIR, snap, 'listings_processed.csv')
        if os.path.exists(file_path):
            try:
                temp = pd.read_csv(file_path, low_memory=False)
                if not temp.empty and 'price_numeric' in temp.columns:
                    df_detail = temp
                    LATEST_SNAPSHOT = snap
                    break
            except:
                continue


# --- 4. VẼ BIỂU ĐỒ ---

# === H1: Xu hướng Nguồn Cung (Bar Chart) ===
if not kpi_supply.empty:
    plt.figure(figsize=(10, 6))
    ax1 = sns.barplot(data=kpi_supply, x='snapshot_date', y='total_listings', color='#1f77b4', width=0.5)
    plt.title('H1. XU HƯỚNG TỔNG NGUỒN CUNG (TOTAL LISTINGS) CỦA THÀNH PHỐ BRUSSELS THEO THỜI GIAN', fontsize=14, fontweight='bold')
    plt.ylabel('TỔNG NGUỒN CUNG (TOTAL LISTINGS)')
    plt.xlabel('THỜI GIAN THU THẬP (SNAPSHOTS)')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    for i in ax1.containers: ax1.bar_label(i, padding=3, fontsize=11, fontweight='bold')
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_01_supply.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H1")


# === H2: Xu hướng Giá (Line/Area Chart) ===
if not kpi_supply.empty:
    plt.figure(figsize=(10, 6))
    plt.plot(kpi_supply['snapshot_date'], kpi_supply['median_price'], marker='o', linewidth=3, color='#2ca02c')
    plt.fill_between(kpi_supply['snapshot_date'], kpi_supply['median_price'], color='#2ca02c', alpha=0.1)
    plt.title('H2. XU HƯỚNG GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE) CỦA THÀNH PHỐ BRUSSELS THEO THỜI GIAN', fontsize=14, fontweight='bold')
    plt.ylabel('GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE - EUR)')
    plt.xlabel('')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    for x, y in zip(kpi_supply['snapshot_date'], kpi_supply['median_price']):
        plt.text(x, y + 2, f"€{y:.0f}", ha='center', va='bottom', fontsize=11, fontweight='bold', color='darkgreen')
    plt.ylim(bottom=0, top=kpi_supply['median_price'].max()*1.2)
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_02_price.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H2")


# === H3: Xu hướng Lấp đầy (Occupancy Rate - Line Chart) ===
if not kpi_season.empty:
    df_h5 = kpi_season.copy()
    # Chuyển đổi ngày tháng
    df_h5['date_dt'] = pd.to_datetime(df_h5['date'], errors='coerce')
    df_h5['snapshot_dt'] = pd.to_datetime(df_h5['snapshot_date'], errors='coerce')
    
    # Lấy dữ liệu từ snapshot MỚI NHẤT
    latest_snap_dt = df_h5['snapshot_dt'].max()
    df_final_h5 = df_h5[df_h5['snapshot_dt'] == latest_snap_dt].sort_values('date_dt')

    if not df_final_h5.empty:
        plt.figure(figsize=(14, 7))
        sns.lineplot(data=df_final_h5, x='date_dt', y='occupancy_rate', color='#d62728', linewidth=2)
        
        # Format trục
        plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
        plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))

        plt.title(f'H3. XU HƯỚNG TỶ LỆ LẤP ĐẦY (OCCUPANCY RATE) CỦA THÀNH PHỐ BRUSSELS THEO SNAPSHOT MỚI NHẤT', 
                  fontsize=14, fontweight='bold')
        plt.ylabel('TỶ LỆ LẤP ĐẦY (OCCUPANCY RATE)')
        plt.xlabel('THỜI GIAN')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.ylim(0, 1.05)
        plt.xticks(rotation=45)
        
        plt.savefig(os.path.join(FIGURES_DIR, 'brussels_03_occupancy.pdf'), format='pdf', bbox_inches='tight')
        plt.close()
        print("   > Đã vẽ H3")
    else:
        print("   ! Không có dữ liệu occupancy cho snapshot mới nhất.")
else:
    print("   ! Không có dữ liệu seasonality.")


# === H4: Cơ cấu Loại phòng (Grouped Bar) ===
if not kpi_room.empty:
    plt.figure(figsize=(12, 6))
    kpi_room['dt'] = pd.to_datetime(kpi_room['snapshot_date'], errors='coerce')
    kpi_room = kpi_room.sort_values('dt')
    
    ax3 = sns.barplot(data=kpi_room, x='snapshot_date', y='percentage', hue='room_type', palette='Set2')
    plt.title('H4. PHÂN BỐ CƠ CẤU LOẠI PHÒNG (ROOM TYPE) CỦA THÀNH PHỐ BRUSSELS THEO THỜI GIAN', fontsize=14, fontweight='bold')
    plt.ylabel('TỶ LỆ (%)')
    plt.xlabel('')
    plt.legend(title='LOẠI PHÒNG (ROOM TYPE)', bbox_to_anchor=(1, 1))
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    for i in ax3.containers: ax3.bar_label(i, fmt='%.1f%%', padding=3, fontsize=9)
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_04_room_type.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H4")


# === H5: Bản đồ Phân bố Listing (Scatter Map) ===
if not df_detail.empty and 'latitude' in df_detail.columns:
    plt.figure(figsize=(12, 10))
    
    # Vẽ điểm (lọc giá < 500)
    sns.scatterplot(
        data=df_detail[df_detail['price_numeric'] < 500], 
        x='longitude', y='latitude', 
        hue='price_numeric', palette='viridis', 
        size='price_numeric', sizes=(10, 100), alpha=0.6,
        legend='brief'
    )

    # Gắn nhãn tên khu vực
    if 'neighbourhood_cleansed' in df_detail.columns:
        neigh_centers = df_detail.groupby('neighbourhood_cleansed')[['latitude', 'longitude']].mean().reset_index()
        for i, row in neigh_centers.iterrows():
            txt = plt.text(
                row['longitude'], row['latitude'], 
                row['neighbourhood_cleansed'], 
                fontsize=9, fontweight='bold', color='black',
                ha='center', va='center'
            )
            txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='white')])

    plt.title(f'H5. BẢN ĐỒ PHÂN BỐ GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE) CỦA THÀNH PHỐ BRUSSELS THEO SNAPSHOT MỚI NHẤT', fontsize=14, fontweight='bold')
    plt.xlabel('LONGITUDE')
    plt.ylabel('LATITUDE')
    plt.legend(title='GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE - EUR)', bbox_to_anchor=(1, 1), loc='upper left')
    plt.axis('equal') 
    
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_05_map_labeled.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H5")


# === H6: TOP 10 KHU VỰC CÓ NHIỀU LISTINGS NHẤT ===
if not df_detail.empty and 'neighbourhood_cleansed' in df_detail.columns:
    top_supply_neigh = df_detail['neighbourhood_cleansed'].value_counts().head(10).reset_index()
    top_supply_neigh.columns = ['neighbourhood_cleansed', 'count']

    plt.figure(figsize=(12, 7))
    sns.barplot(
        data=top_supply_neigh,
        y='neighbourhood_cleansed',
        x='count',
        palette='Blues_r',
        hue='neighbourhood_cleansed',
        legend=False
    )
    plt.title(f'H6. TOP 10 KHU VỰC CÓ TỔNG NGUỒN CUNG (TOTAL LISTINGS) CAO NHẤT THÀNH PHỐ BRUSSELS THEO SNAPSHOT MỚI NHẤT', fontsize=14, fontweight='bold')
    plt.xlabel('TỔNG NGUỒN CUNG (TOTAL LISTINGS)')
    plt.ylabel('')
    for i, v in enumerate(top_supply_neigh['count']):
        plt.text(v + 10, i, f"{v}", va='center', fontweight='bold')
        
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_06_top10_supply.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H6")


# === H7: Top 10 Khu vực Đắt đỏ nhất (Bar Chart) ===
if not df_detail.empty and 'neighbourhood_cleansed' in df_detail.columns:
    neigh_price = df_detail.groupby('neighbourhood_cleansed')['price_numeric'].median().sort_values(ascending=False).head(10).reset_index()

    plt.figure(figsize=(12, 7))
    sns.barplot(
        data=neigh_price, 
        y='neighbourhood_cleansed', 
        x='price_numeric', 
        palette='Reds_r', 
        hue='neighbourhood_cleansed', 
        legend=False
    )
    plt.title(f'H7. TOP 10 KHU VỰC CÓ GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE) CAO NHẤT THÀNH PHỐ BRUSSELS THEO SNAPSHOT MỚI NHẤT', fontsize=14, fontweight='bold')
    plt.xlabel('GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE - EUR)')
    plt.ylabel('')
    for i, v in enumerate(neigh_price['price_numeric']):
        plt.text(v + 1, i, f"€{v:.0f}", va='center', fontweight='bold')
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_07_top10_price.pdf'), format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H7")


# === H8: PHÂN PHỐI GIÁ THUÊ (HISTOGRAM + KDE) ===
if not df_detail.empty and 'price_numeric' in df_detail.columns:
    plt.figure(figsize=(12, 6))
    
    sns.histplot(
        df_detail[df_detail['price_numeric'] < 500]['price_numeric'],
        kde=True,
        bins=40,
        color='#1f77b4',
        alpha=0.7
    )
    
    plt.title(f'H8. PHÂN PHỐI GIÁ THUÊ (PRICE DISTRIBUTION) DỰA TRÊN TỔNG NGUỒN CUNG (TOTAL LISTINGS) CỦA THÀNH PHỐ BRUSSELS THEO SNAPSHOT MỚI NHẤT',
              fontsize=14, fontweight='bold')
    plt.xlabel('GIÁ THUÊ (EUR)')
    plt.ylabel('TỔNG NGUỒN CUNG (TOTAL LISTINGS)')
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    
    plt.savefig(os.path.join(FIGURES_DIR, 'brussels_08_price_distribution.pdf'), 
                format='pdf', bbox_inches='tight')
    plt.close()
    print("   > Đã vẽ H8")


# === H9: HEATMAP BIẾN ĐỘNG GIÁ THEO KHU VỰC VÀ THỜI GIAN ===
all_files = glob.glob(os.path.join(PROCESSED_DIR, '*', 'listings_processed.csv'))
if all_files:
    df_list = []
    for f in all_files:
        snap_name = os.path.basename(os.path.dirname(f))
        try:
            d = pd.read_csv(f, usecols=['neighbourhood_cleansed', 'price_numeric'], low_memory=False)
            d['snapshot'] = snap_name
            df_list.append(d)
        except: pass
    
    if df_list:
        df_heatmap = pd.concat(df_list, ignore_index=True)
        top_neighs = df_heatmap['neighbourhood_cleansed'].value_counts().head(19).index
        df_heatmap = df_heatmap[df_heatmap['neighbourhood_cleansed'].isin(top_neighs)]
        
        heatmap_data = df_heatmap.groupby(['neighbourhood_cleansed', 'snapshot'])['price_numeric'].median().unstack()
        sorted_cols = sorted(heatmap_data.columns, key=parse_snapshot_name)
        heatmap_data = heatmap_data[sorted_cols]

        plt.figure(figsize=(12, 8))
        sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlGnBu', linewidths=.5, cbar_kws={'label': 'GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE)'})
        plt.title('H9. BIẾN ĐỘNG GIÁ THUÊ TRUNG VỊ (MEDIAN PRICE) CỦA THÀNH PHỐ BRUSSELS THEO KHU VỰC VÀ THỜI GIAN', fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('THỜI ĐIỂM (SNAPSHOT)'); plt.ylabel('KHU VỰC (NEIGHBOURHOOD)')
        plt.savefig(os.path.join(FIGURES_DIR, 'brussels_09_price_heatmap.pdf'), format='pdf', bbox_inches='tight')
        plt.close()
        print("   > Đã vẽ H9")


--- BẮT ĐẦU VẼ CHUYÊN SÂU CHO BRUSSELS ---
--- ĐANG NẠP DỮ LIỆU TỔNG HỢP ---
   > Đã vẽ H1
   > Đã vẽ H2
   > Đã vẽ H3
   > Đã vẽ H4
   > Đã vẽ H5
   > Đã vẽ H6
   > Đã vẽ H7
   > Đã vẽ H8
   > Đã vẽ H9
